# Brain Tumor Classification

## Loading the Data

In [ ]:
import kagglehub
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

## Installing Libraries

In [ ]:
!pip install torchmetrics
!pip install torchsummary
!pip install tqdm

## Data Preparation

In [ ]:
import os
os.listdir(path)

In [ ]:
test_path = os.path.join(path, "Testing")
train_path = os.path.join(path, "Training")
classnames = os.listdir(train_path)
classnames

In [ ]:
for classname in classnames:
    print(classname, len(os.listdir(os.path.join(test_path, classname))))

In [ ]:
for classname in classnames:
    print(classname, len(os.listdir(os.path.join(train_path, classname))))

### Creating the dataframe

In [ ]:
# create a dataframe with image_path and image_class
import pandas as pd

train_df = []
test_df = []
for classname in classnames:
    for image in os.listdir(os.path.join(train_path, classname)):
        image_path = os.path.join(train_path, classname, image)
        train_df.append({'image_path': image_path, 'image_class': classname})

for classname in classnames:
    for image in os.listdir(os.path.join(test_path, classname)):
        image_path = os.path.join(test_path, classname, image)
        test_df.append({'image_path': image_path, 'image_class': classname})


train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
train_df.shape, test_df.shape

In [ ]:
train_df = train_df[train_df['image_path'].str.endswith(('.jpg', '.png'))]
test_df = test_df[test_df['image_path'].str.endswith(('.jpg', '.png'))]
train_df.shape, test_df.shape

### Visualising Class distribution

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(ncols=2)
train_df['image_class'].value_counts().plot(kind='pie', ax=axs[0])
test_df['image_class'].value_counts().plot(kind='pie', ax=axs[1])
axs[0].set_title('Train set')
axs[1].set_title('Test Set')
fig.suptitle('Brain tumor class distribution')
plt.show()


### Encoding the class

Encoding the class using label encoder from sklearn

In [ ]:
import joblib
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the 'image_class' column
train_df['encoded_class'] = label_encoder.fit_transform(train_df['image_class'])
test_df['encoded_class'] = label_encoder.transform(test_df['image_class'])

joblib.dump(label_encoder, "label_encoder.joblib")

# Print the mapping between original class names and encoded labels
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

### Visualising Few Sample Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms.v2 as v2

test_transform = v2.Compose([
    v2.Grayscale(num_output_channels=3),
    v2.Resize((224, 224)),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(10),
    v2.RandomAdjustSharpness(sharpness_factor=5, p=0.5),
    v2.RandomAutocontrast(p=0.5),
    v2.ToImage(),
])
sample_imgs = np.random.randint(low=0, high=len(train_df),size=5)
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(8,10))


for i in range(5):    
        img_path, img_class, _ = train_df.iloc[sample_imgs[i]]
        img = Image.open(img_path).convert('RGB')
        img_transformed = test_transform(img).permute(1,2,0).numpy()
        
        axs[i,0].imshow(img)
        axs[i,0].set_title(f'original: image class = {img_class}')
        
        axs[i,0].axis('off')

        axs[i,1].imshow(img_transformed)
        axs[i,1].set_title(f'transformed: image class = {img_class}')
        axs[i,1].axis('off')

fig.suptitle('Visualising the sample images')
plt.show()

### Creating the Dataset

Creating the pytorch custom dataset and respective dataloaders

In [ ]:
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms.v2 as v2

torch.manual_seed(42)
torch.cuda.manual_seed(42)
class TumorDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        img_class = self.dataframe.iloc[idx]['encoded_class']

        image = Image.open(img_path)
        image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_class


transform = v2.Compose([
    v2.Grayscale(num_output_channels=3),
    v2.Resize((224, 224)),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(10),
    v2.RandomAdjustSharpness(sharpness_factor=5, p=0.5),
    v2.RandomAutocontrast(p=0.5),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = TumorDataset(train_df, transform=transform)
test_dataset = TumorDataset(test_df, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

# set torch random seed

# Create data loaders for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Training


### Machine Learning Models

1. SVM
2. Random Forest
3. XGBoost


**Extracting Feature Vector from ResNet 18 model**


In [ ]:
import torch.nn as nn
import torchvision.models as models


# Load pre-trained ResNet18 model
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Remove the final classification layer
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Identity()



# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

# Extract feature vectors for training data
train_features = []
train_labels = []
resnet18.eval()  # Set model to evaluation mode
with torch.no_grad():
  for images, labels in train_loader:
    images = images.to(device)
    features = resnet18(images)
    train_features.extend(features.cpu().numpy())
    train_labels.extend(labels)

# Extract feature vectors for testing data
test_features = []
test_labels = []
with torch.no_grad():
  for images, labels in test_loader:
    images = images.to(device)
    features = resnet18(images)
    test_features.extend(features.cpu().numpy())
    test_labels.extend(labels)

# Convert lists to numpy arrays
train_features = np.array(train_features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
test_labels = np.array(test_labels)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

classification_report = []

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)
svm_predictions = svm_classifier.predict(test_features)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
svm_precision = precision_score(test_labels, svm_predictions, average='weighted')
svm_recall = recall_score(test_labels, svm_predictions, average='weighted')
svm_f1 = f1_score(test_labels, svm_predictions, average='weighted')
classification_report.append({'model': 'SVM', 'accuracy': svm_accuracy, 'precision': svm_precision, 'recall': svm_recall, 'f1': svm_f1})

# Train Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(train_features, train_labels)
rf_predictions = rf_classifier.predict(test_features)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
rf_precision = precision_score(test_labels, rf_predictions, average='weighted')
rf_recall = recall_score(test_labels, rf_predictions, average='weighted')
rf_f1 = f1_score(test_labels, rf_predictions, average='weighted')
classification_report.append({'model': 'Random Forest', 'accuracy': rf_accuracy, 'precision': rf_precision, 'recall': rf_recall, 'f1': rf_f1})

# Train XGBoost classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(train_features, train_labels)
xgb_predictions = xgb_classifier.predict(test_features)
xgb_accuracy = accuracy_score(test_labels, xgb_predictions)
xgb_precision = precision_score(test_labels, xgb_predictions, average='weighted')
xgb_recall = recall_score(test_labels, xgb_predictions, average='weighted')
xgb_f1 = f1_score(test_labels, xgb_predictions, average='weighted')
classification_report.append({'model': 'XGBoost', 'accuracy': xgb_accuracy, 'precision': xgb_precision, 'recall': xgb_recall, 'f1': xgb_f1})

joblib.dump(svm_classifier, "svm_classifier.joblib")
joblib.dump(rf_classifier, "rf_classifier.joblib")
joblib.dump(xgb_classifier, "xgb_classifier.joblib")

clf_df = pd.DataFrame(classification_report)
clf_df.to_csv('clf_report.csv')
clf_df.head()

## Creating a Training Module

In [ ]:
import pandas as pd
from tqdm import tqdm
from torchmetrics import Accuracy, Precision, Recall, F1Score
class TrainingModule:
    def __init__(self, model: nn.Module, device, train_loader, test_loader):
        self.model = model
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader

        self.results = []

    def train_step(self, epoch, num_epochs, criterion, optimizer):
        self.model.train()
        running_loss = 0.0
        for images, labels in tqdm(self.train_loader, desc=f'Training: {epoch}/{num_epochs}'):
            images = images.to(self.device)
            labels = labels.to(self.device)
            optimizer.zero_grad()
            outputs = self.model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        running_loss = running_loss / len(self.train_loader)
        return running_loss

    def test_step(self, epoch, num_epochs, criterion):
        self.model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for images, labels in tqdm(self.test_loader, desc=f'Testing: {epoch}/{num_epochs}'):
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()

        running_loss = running_loss / len(self.test_loader)
        return running_loss

    def train(self, criterion, optimizer, num_epochs, save_step):
        for epoch in range(1,num_epochs+1):
            train_loss = self.train_step(epoch, num_epochs, criterion, optimizer)
            test_loss = self.test_step(epoch, num_epochs, criterion)

            self.results.append({'epoch': epoch, 
                                 'train_loss': train_loss,
                                 'test_loss': test_loss
                                })
            
            print(f'Epoch {epoch}/{num_epochs} - Train Loss: {train_loss:.4f} - Test Loss: {test_loss:.4f}')
            if epoch % save_step == 0:
                self.save_model()
                self.get_results()

        self.save_model()
        self.get_results()

    def get_classification_report(self):

        # load the state_dict from model_path,
        self.model.eval()

        accuracy = Accuracy(task='multiclass', num_classes=len(classnames)).to(self.device)
        precision = Precision(task='multiclass',average='weighted', num_classes=len(classnames)).to(self.device)
        recall = Recall(task='multiclass',average='weighted', num_classes=len(classnames)).to(self.device)
        f1 = F1Score(task='multiclass',average='weighted', num_classes=len(classnames)).to(self.device)
        with torch.no_grad():
            for images, labels in tqdm(self.test_loader,
                                       desc=f'Generating Classification Report'):
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)
                accuracy.update(outputs, labels)
                precision.update(outputs, labels)
                recall.update(outputs, labels)
                f1.update(outputs, labels)
        
        return {
                'model': self.model.__class__.__name__,
                'accuracy': accuracy.compute().cpu().item(),
                'precision': precision.compute().cpu().item(),
                'recall': recall.compute().cpu().item(),
                'f1': f1.compute().cpu().item()
                }


    def get_results(self):
        results_df  = pd.DataFrame(self.results)
        results_df.to_csv(f'{self.model.__class__.__name__}_results.csv',
                          index=False)
        return results_df

    def save_model(self):
        torch.save(self.model.state_dict(), 
                   f'{self.model.__class__.__name__}.pth')
        

### Transfer Learning

#### Transfer Learning with ResNet 18 Model

In [ ]:
class ResNet18Model(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18Model, self).__init__()
        self.base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),     
            nn.Softmax(dim=1)
        )
       

    def forward(self, x):
        return self.base_model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Instantiate the model, loss function, and optimizer
resnet18_model = ResNet18Model(len(classnames)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet18_model.parameters(), lr=0.0001)

# Move the model to the device
resnet18_model = resnet18_model.to(device)

# Create a training module
resnet_training_module = TrainingModule(resnet18_model, device, train_loader, test_loader)


In [ ]:
resnet_training_module.train(criterion, optimizer, num_epochs=20, save_step=5)

In [ ]:
# Generate classification report
resnet_results = resnet_training_module.get_results()
resnet_clf_report = resnet_training_module.get_classification_report()
classification_report.append(resnet_clf_report)
clf_df = pd.DataFrame(classification_report)
clf_df.head()

In [ ]:
clf_df.to_csv('clf_report.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(resnet_results['epoch'],resnet_results['train_loss'], label='train loss')
plt.plot(resnet_results['epoch'],resnet_results['test_loss'], label='test loss')
plt.title('Resnet loss curves')
plt.legend(labels=['train loss', 'test loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

#### Transfer Learning with DenseNet Model

In [ ]:
class DenseNetModel(nn.Module):
    
    def __init__(self, num_classes):
        super(DenseNetModel, self).__init__()
        # Load pre-trained DenseNet121 model
        self.base_model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

        # Replace the classifier with a new one for our number of classes
        num_ftrs = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.base_model(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Instantiate the model, loss function, and optimizer
densenet_model = DenseNetModel(len(classnames)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(densenet_model.parameters(), lr=0.001)

# Move the model to the device
densenet_model = densenet_model.to(device)

# Create a training module
densenet_training_module = TrainingModule(densenet_model, device, train_loader, test_loader)


In [ ]:
densenet_training_module.train(criterion, optimizer, num_epochs=20, save_step=5)

In [ ]:
# Generate classification report
densenet_results = densenet_training_module.get_results()
classification_report_new = densenet_training_module.get_classification_report()
classification_report.append(classification_report_new)
clf_df = pd.DataFrame(classification_report)
clf_df.head()

In [ ]:
clf_df.to_csv('clf_report.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(densenet_results['epoch'],densenet_results['train_loss'], label='train loss')
plt.plot(densenet_results['epoch'],densenet_results['test_loss'], label='test loss')
plt.title('DenseNet loss curves')
plt.legend(labels=['train loss', 'test loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

#### Transfer Learning with EfficentNetB0 Model

In [ ]:
class EFNetModel(nn.Module):
    def __init__(self, num_classes):
        super(EFNetModel, self).__init__()
        self.base_model = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.DEFAULT
        )
        num_ftrs = self.base_model.classifier[1].in_features
        self.base_model.classifier[1] = nn.Sequential(
            nn.Linear(self.base_model.classifier[1].in_features, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.base_model(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Instantiate the model, loss function, and optimizer
efnet_model = EFNetModel(len(classnames)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(efnet_model.parameters(), lr=0.001)

# Move the model to the device
efnet_model = efnet_model.to(device)

# Create a training module
efnet_training_module = TrainingModule(efnet_model,
                                            device, train_loader, test_loader)



In [ ]:
efnet_training_module.train(criterion, optimizer, num_epochs=10, save_step=5)

In [ ]:
# Generate classification report
efnet_results = efnet_training_module.get_results()
ef_clf_report = efnet_training_module.get_classification_report()
classification_report.append(ef_clf_report)
clf_df = pd.DataFrame(classification_report)
clf_df.tail()

In [ ]:
clf_df.to_csv('clf_report.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(efnet_results['epoch'],efnet_results['train_loss'], label='train loss')
plt.plot(efnet_results['epoch'],efnet_results['test_loss'], label='test loss')
plt.title('EFNet loss curves')
plt.legend(labels=['train loss', 'test loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

### Deep Learning

Deep Learning using convolutional layers

In [ ]:
import torch.nn.functional as F

class BTModel(nn.Module):
    def __init__(self, num_classes):
        super(BTModel, self).__init__()

        self.features = nn.Sequential(
           nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes),
            nn.Softmax(dim=1)
        )
        
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Instantiate the model, loss function, and optimizer
bt_model = BTModel(len(classnames)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(bt_model.parameters(), lr=0.001)

# Move the model to the device
bt_model = bt_model.to(device)

# Create a training module
bt_training_module = TrainingModule(bt_model, device, train_loader, test_loader)

In [ ]:
bt_training_module.train(criterion, optimizer, num_epochs=20, save_step=5)

In [ ]:
# Generate classification report
bt_results = bt_training_module.get_results()
bt_clf_report = bt_training_module.get_classification_report()
classification_report = []
classification_report.append(bt_clf_report)
clf_df = pd.DataFrame(classification_report)
clf_df.tail()

In [ ]:
clf_df.to_csv('clf_report.csv')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(bt_results['epoch'],bt_results['train_loss'], label='train loss')
plt.plot(bt_results['epoch'],bt_results['test_loss'], label='test loss')
plt.title('BT loss curves')
plt.legend(labels=['train loss', 'test loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

### Prediction

In [ ]:
import joblib
import torch


label_encoder = joblib.load('label_encoder.joblib')
rf_classifier = joblib.load('rf_classifier.joblib')
svm_classifier = joblib.load('svm_classifier.joblib')
xgb_classifier = joblib.load('xgb_classifier.joblib')

resnet_model = ResNet18Model(len(classnames)).to(device)
resnet_model.load_state_dict(torch.load("./ResNet18Model.pth",
                                            map_location=device,
                                            weights_only=False))

densenet_model = DenseNetModel(len(classnames)).to(device)
densenet_model.load_state_dict(torch.load("./DenseNetModel.pth",
                                         map_location=device,
                                         weights_only=False))

efnet_model = EFNetModel(len(classnames)).to(device)
efnet_model.load_state_dict(torch.load("./EFNetModel.pth",
                                           map_location=device,
                                           weights_only=False))

bt_model = BTModel(len(classnames)).to(device)
bt_model.load_state_dict(torch.load("./BTModel.pth",
                                        map_location=device,
                                        weights_only=False))

def predict(image):
    densenet_model.eval()
    efnet_model.eval()
    bt_model.eval()
    resnet_model.eval()
    
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(device)
        resnet_class = torch.argmax(resnet_model(image_tensor))
        densenet_class = torch.argmax(densenet_model(image_tensor))
        efnet_class = torch.argmax(efnet_model(image_tensor))
        bt_class = torch.argmax(bt_model(image_tensor))
        
        image_numpy = resnet18(image_tensor)
        image_numpy = image_numpy.cpu().numpy()
    
    rf_class = rf_classifier.predict(image_numpy)
    svm_class = svm_classifier.predict(image_numpy)
    xgb_class = xgb_classifier.predict(image_numpy)

    results = {
        'resnet': resnet_class.item(),
        'densenet': densenet_class.item(),
        'efnet': efnet_class.item(),
        'bt': bt_class.item(),
        'rf': rf_class[0],
        'svm': svm_class[0],
        'xgb': xgb_class[0]        
    }

    predicted_classes = np.array(list(results.values()))
    unique_values, counts = np.unique(predicted_classes, return_counts=True)

    # # Find the index of the maximum count
    max_count_index = np.argmax(counts)

    # # Get the number that appears the most
    most_frequent_number = unique_values[max_count_index]
    
    predicted_class = label_encoder.inverse_transform([most_frequent_number])[0]
    results['resnet'] = label_encoder.inverse_transform([results['resnet']])[0]
    results['densenet'] = label_encoder.inverse_transform([results['densenet']])[0]
    results['efnet'] = label_encoder.inverse_transform([results['efnet']])[0]
    results['bt'] = label_encoder.inverse_transform([results['bt']])[0]
    results['rf'] = label_encoder.inverse_transform([results['rf']])[0]
    results['xgb'] = label_encoder.inverse_transform([results['xgb']])[0]
    results['svm'] = label_encoder.inverse_transform([results['svm']])[0]
    results['final'] = predicted_class
    
    return predicted_class, most_frequent_number, results
    

In [ ]:
import random
import numpy as np
from PIL import Image

sample_imgs = np.random.randint(low=0, high=len(train_df),size=10)
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(10,10))
k = 0
for i in range(5):
    for j in range(2):
        img_path, img_class, _ = train_df.iloc[sample_imgs[k]]
        img = Image.open(img_path).convert('RGB')
        img_transformed = transform(img)
        predicted_class,_,results = predict(img_transformed)
        axs[i,j].imshow(img)
        axs[i,j].set_title(f'original={img_class}, predicted={predicted_class}')
        axs[i,j].axis('off')
        k+=1
        
fig.suptitle('Visualising the sample images')
plt.show()

In [ ]:
import torch
def generate_classification_report():
    accuracy = Accuracy(task='multiclass', num_classes=len(classnames)).to(device)
    precision = Precision(task='multiclass',average='weighted', num_classes=len(classnames)).to(device)
    recall = Recall(task='multiclass',average='weighted', num_classes=len(classnames)).to(device)
    f1 = F1Score(task='multiclass',average='weighted', num_classes=len(classnames)).to(device)

    for images, labels in tqdm(test_loader, desc='Generating classification report'):
        outputs = []
        images = images.to(device)
        labels = labels.to(device)
        for image in images:
            
            outputs.append(int(predict(image)[1]))

        outputs = torch.tensor(outputs).to(device)

        accuracy.update(outputs, labels)
        precision.update(outputs, labels)
        recall.update(outputs, labels)
        f1.update(outputs, labels)

    return {
            'model': "final",
            'accuracy': accuracy.compute().cpu().item(),
            'precision': precision.compute().cpu().item(),
            'recall': recall.compute().cpu().item(),
            'f1': f1.compute().cpu().item()
        }


final_clf_report = generate_classification_report()
classification_report.append(final_clf_report)
clf_df = pd.DataFrame(classification_report)
clf_df

In [ ]:
clf_report